# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-f339c67936-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 

"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Civilization explorative search") 

Consider the following exploratory information need:

> Investigate the Ancient Egypt and the related archeological explorations and findings.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q13444093`| Meidum Pyramid    | node |
| `wd:Q41127`| Poseidon   | node |





Also consider

```
wd:Q13444093 ?p ?obj .
```

is the BGP to retrieve all **properties of Meidum Pyramid**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for Ancient Egypt

2. Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

3. Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

4. Identify the BGP for archeologist.

5. For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

6. Find the BGP for deity.

7. List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

8. Compare ancient Greece with ancient Egypt and find how many deities each one has.


## Task 1
Identify the BGP for Ancient Egypt

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q13444093 ?p ?obj .
   # get the label
   ?p sc:name ?name.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1481'), ('name', 'vici.org ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1566'), ('name', 'GeoNames ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material')]
[('p', 'http://www.wikidata.org/prop/direct/P2044'), ('name', 'elevation above sea level')]
[('p', 'http://www.wikidata.org/prop/direct/P2048'), ('name', 'height')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2326'), ('name', 'GNS Unique Feature ID')]
[('p', 'http://www.wi

In [6]:
#wd:Q13444093	Meidum Pyramid
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q13444093 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5008'), ('name', 'on focus list of Wikimedia project'), ('obj', 'http://www.wikidata.org/entity/Q68074438'), ('objName', 'WikiProject Livius')]
[('p', 'http://www.wikidata.org/prop/direct/P88'), ('name', 'commissioned by'), ('obj', 'http://www.wikidata.org/entity/Q189371'), ('objName', 'Sneferu')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location'), ('obj', 'http://www.wikidata.org/entity/Q1066424'), ('objName', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after'), ('obj', 'http://www.wikidata.org/entity/Q1066424'), ('objName', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P88'), ('name', 'commissioned by'), ('obj', 'http://www.wikidata.org/entity/Q220240'), ('objName', 'Huni')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material'), ('obj', 'http://www.wikidata.org/entity/Q23757'), ('objName', 'limestone')]
[('p', 'http://www.wikidata.org/pro

In [9]:
# Q189371 'Sneferu'
# Q79 Egypt
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q189371 ?p ?obj  .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   filter regex(?objName, "Ancient Egypt")
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period'), ('obj', 'http://www.wikidata.org/entity/Q11768'), ('objName', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship'), ('obj', 'http://www.wikidata.org/entity/Q11768'), ('objName', 'Ancient Egypt')]
2


In [13]:
# write your final query
#'P2348' 'time period'
# Q189371 'Sneferu'
# Q79 Egypt
queryString = """
SELECT DISTINCT ?period ?periodTime
WHERE {
    
   wd:Q189371 ?p ?obj.
   ?obj wdt:P2348 ?period.
   
   ?period sc:name ?periodTime.
   filter regex(?periodTime, "Ancient Egypt")
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('period', 'http://www.wikidata.org/entity/Q11768'), ('periodTime', 'Ancient Egypt')]
1


Final query for this task

## Task 2
Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

In [18]:

#trovare la lista dei faraoni
#Q189371 'Sneferu' that is a pharaohs
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q189371 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   optional{?obj sc:name ?objName.}
   
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child'), ('obj', 'http://www.wikidata.org/entity/Q122426'), ('objName', 'Henutsen')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child'), ('obj', 'http://www.wikidata.org/entity/Q161904'), ('objName', 'Khufu')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period'), ('obj', 'http://www.wikidata.org/entity/Q177819'), ('objName', 'Old Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P53'), ('name', 'family'), ('obj', 'http://www.wikidata.org/entity/Q211001'), ('objName', 'Fourth Dynasty of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse'), ('obj', 'http://www.wikidata.org/entity/Q256515'), ('objName', 'Hetepheres I')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('name', 'sibling'), ('obj', 'http://www.wikidata.org/entity/Q256515'), ('objName', 'Hetepheres I')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child'), 

In [25]:
# P569  'date of birth' 
# P39 'position held' Q37110 'pharaoh'
queryString = """
SELECT DISTINCT ?pharaohs ?name ?dateBirth
WHERE {
   # bind something
   ?pharaohs wdt:P39 wd:Q37110;
       wdt:P569 ?dateBirth.
   
   # get the label
   ?pharaohs sc:name ?name.
   filter (?dateBirth > "-1500-01-01T00:00:00"^^xsd:dateTime && ?dateBirth < "-1000-01-01T00:00:00"^^xsd:dateTime).
   
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pharaohs', 'http://www.wikidata.org/entity/Q336395'), ('name', 'Smendes'), ('dateBirth', '-1200-01-01T00:00:00Z')]
[('pharaohs', 'http://www.wikidata.org/entity/Q380676'), ('name', 'Psusennes I'), ('dateBirth', '-1100-01-01T00:00:00Z')]
[('pharaohs', 'http://www.wikidata.org/entity/Q434877'), ('name', 'Amenemnisu'), ('dateBirth', '-1100-01-01T00:00:00Z')]
[('pharaohs', 'http://www.wikidata.org/entity/Q705367'), ('name', 'Menkheperre'), ('dateBirth', '-1100-01-01T00:00:00Z')]
[('pharaohs', 'http://www.wikidata.org/entity/Q459153'), ('name', 'Osorkon II'), ('dateBirth', '-1000-01-01T00:00:00Z')]
[('pharaohs', 'http://www.wikidata.org/entity/Q146029'), ('name', 'Ay'), ('dateBirth', '-1400-01-01T00:00:00Z')]
[('pharaohs', 'http://www.wikidata.org/entity/Q152230'), ('name', 'Tiye'), ('dateBirth', '-1397-01-01T00:00:00Z')]
[('pharaohs', 'http://www.wikidata.org/entity/Q153344'), ('name', 'Meritaten'), ('dateBirth', '-1400-01-01T00:00:00Z')]
[('pharaohs', 'http://www.wikidata.org/e

Final query for this task

In [33]:
# P569  'date of birth' 
# P39 'position held' Q37110 'pharaoh'
# P40 'child'
queryString = """
SELECT DISTINCT ?pharaohs ?name (count(?child) as ?numChild)
WHERE {
   # bind something
   ?pharaohs wdt:P39 wd:Q37110;
       wdt:P569 ?dateBirth;
       wdt:P40 ?child.
   
   # get the label
   ?pharaohs sc:name ?name.
   filter (?dateBirth > "-1500-01-01T00:00:00"^^xsd:dateTime && ?dateBirth < "-1000-01-01T00:00:00"^^xsd:dateTime).
   
   
}
GROUP BY ?pharaohs ?name 
HAVING (count (?child) > 8)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pharaohs', 'http://www.wikidata.org/entity/Q1528'), ('name', 'Ramesses III'), ('numChild', '10')]
[('pharaohs', 'http://www.wikidata.org/entity/Q1523'), ('name', 'Ramesses II'), ('numChild', '18')]
[('pharaohs', 'http://www.wikidata.org/entity/Q42606'), ('name', 'Amenhotep III'), ('numChild', '10')]
[('pharaohs', 'http://www.wikidata.org/entity/Q152230'), ('name', 'Tiye'), ('numChild', '9')]
[('pharaohs', 'http://www.wikidata.org/entity/Q81794'), ('name', 'Akhenaten'), ('numChild', '9')]
5


## Task 3
Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

In [138]:
#P2348 'time period' --> Q11768 'Ancient Egypt'

queryString = """
 

SELECT DISTINCT ?p ?name ?obj
WHERE {
   # bind something
   wd:Q11768 ?p ?obj .
   # get the label
   ?p sc:name ?name.
  
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification'), ('obj', '932')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification'), ('obj', '2--32')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q8440998')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key'), ('obj', 'Tag:historic:civilization=ancient_egyptian')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q1138524')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q17378135')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q4173137')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name

In [149]:
#P2348 'time period' --> Q11768 'Ancient Egypt'
#look for "instance of"

queryString = """
 

SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q11768 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   filter regex(?name, "instance of").
  
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28171280'), ('objName', 'ancient civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q3024240'), ('objName', 'historical country')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q3502482'), ('objName', 'cultural region')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1620908'), ('objName', 'historical region')]
4


In [150]:
#Q28171280 'ancient civilization' 
#P31 'instance of' 
queryString = """
 

SELECT DISTINCT ?p ?name ?obj
WHERE {
   # bind something
   ?p wdt:P31 wd:Q28171280 .
   # get the label
   ?p sc:name ?name.
  

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q35355'), ('name', 'Sumer')]
[('p', 'http://www.wikidata.org/entity/Q25558247'), ('name', 'Chinese civilization')]
[('p', 'http://www.wikidata.org/entity/Q38060'), ('name', 'Gaul')]
[('p', 'http://www.wikidata.org/entity/Q134178'), ('name', 'Minoan civilization')]
[('p', 'http://www.wikidata.org/entity/Q173082'), ('name', 'Dacia')]
[('p', 'http://www.wikidata.org/entity/Q2429397'), ('name', 'Ancient Carthage')]
[('p', 'http://www.wikidata.org/entity/Q3327860'), ('name', 'Celtic kingdom of Ireland')]
[('p', 'http://www.wikidata.org/entity/Q193748'), ('name', 'history of ancient Israel and Judah')]
[('p', 'http://www.wikidata.org/entity/Q11768'), ('name', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece')]
[('p', 'http://www.wikidata.org/entity/Q41741'), ('name', 'Thrace')]
[('p', 'http://www.wikidata.org/entity/Q128904'), ('name', 'Elam')]
[('p', 'http://www.wikidata.org/entity/Q185068'), ('name', 

In [165]:
#Q28171280 'ancient civilization' 
#P31 'instance of' 
#for duration we make  #P582 'end time' - #P580 'start time' 

queryString = """
 

SELECT DISTINCT ?p ?name ?obj ?duration
WHERE {
   # bind something
   ?p wdt:P31 wd:Q28171280.
    OPTIONAL{
        ?p wdt:P582 ?endTime.
        ?p wdt:P580 ?startTime.
        bind(year(?endTime)-year(?startTime) as ?duration )
    }
   # get the label
   ?p sc:name ?name.
   
  

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q134178'), ('name', 'Minoan civilization'), ('duration', '2200')]
[('p', 'http://www.wikidata.org/entity/Q134178'), ('name', 'Minoan civilization'), ('duration', '901')]
[('p', 'http://www.wikidata.org/entity/Q11768'), ('name', 'Ancient Egypt'), ('duration', '4395')]
[('p', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece'), ('duration', '1800')]
[('p', 'http://www.wikidata.org/entity/Q42534'), ('name', 'Indus Valley Civilization'), ('duration', '1200')]
[('p', 'http://www.wikidata.org/entity/Q1551054'), ('name', 'Israelite period'), ('duration', '614')]
[('p', 'http://www.wikidata.org/entity/Q35355'), ('name', 'Sumer')]
[('p', 'http://www.wikidata.org/entity/Q25558247'), ('name', 'Chinese civilization')]
[('p', 'http://www.wikidata.org/entity/Q38060'), ('name', 'Gaul')]
[('p', 'http://www.wikidata.org/entity/Q173082'), ('name', 'Dacia')]
[('p', 'http://www.wikidata.org/entity/Q2429397'), ('name', 'Ancient Carthage')]
[('

Final query for this task

In [172]:
# write your final query
# Q28171280 'ancient civilization' 
# P31 'instance of' 
#for duration we make  #P582 'end time' - #P580 'start time' 

queryString = """
 

SELECT DISTINCT ?p ?name ?obj ?duration
WHERE {
   # bind something
   ?p wdt:P31 wd:Q28171280.
    OPTIONAL{
        ?p wdt:P582 ?endTime.
        ?p wdt:P580 ?startTime.
        bind(year(?endTime)-year(?startTime) as ?duration )
    }
   # get the label
   ?p sc:name ?name.   
  

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q134178'), ('name', 'Minoan civilization'), ('duration', '2200')]
[('p', 'http://www.wikidata.org/entity/Q134178'), ('name', 'Minoan civilization'), ('duration', '901')]
[('p', 'http://www.wikidata.org/entity/Q11768'), ('name', 'Ancient Egypt'), ('duration', '4395')]
[('p', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece'), ('duration', '1800')]
[('p', 'http://www.wikidata.org/entity/Q42534'), ('name', 'Indus Valley Civilization'), ('duration', '1200')]
[('p', 'http://www.wikidata.org/entity/Q1551054'), ('name', 'Israelite period'), ('duration', '614')]
[('p', 'http://www.wikidata.org/entity/Q35355'), ('name', 'Sumer')]
[('p', 'http://www.wikidata.org/entity/Q25558247'), ('name', 'Chinese civilization')]
[('p', 'http://www.wikidata.org/entity/Q38060'), ('name', 'Gaul')]
[('p', 'http://www.wikidata.org/entity/Q173082'), ('name', 'Dacia')]
[('p', 'http://www.wikidata.org/entity/Q2429397'), ('name', 'Ancient Carthage')]
[('

## Task 4
Identify the BGP for archeologist.

In [176]:
# write your queries
# wd:Q5
# wdt:P31	instance of
# wdt:P106	occupation

queryString = """
SELECT DISTINCT ?p ?name ?occupation
WHERE {
   # bind something
   ?p ?P31 wd:Q5 ;
        wdt:P106 ?occupation.
       
   # get the label
   ?p sc:name ?name.
   

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1002728'), ('name', 'Fonsi Nieto'), ('occupation', 'http://www.wikidata.org/entity/Q378622')]
[('p', 'http://www.wikidata.org/entity/Q1002728'), ('name', 'Fonsi Nieto'), ('occupation', 'http://www.wikidata.org/entity/Q10349745')]
[('p', 'http://www.wikidata.org/entity/Q1002728'), ('name', 'Fonsi Nieto'), ('occupation', 'http://www.wikidata.org/entity/Q3014296')]
[('p', 'http://www.wikidata.org/entity/Q100343342'), ('name', 'Lubomír Stoklásek'), ('occupation', 'http://www.wikidata.org/entity/Q43845')]
[('p', 'http://www.wikidata.org/entity/Q100343342'), ('name', 'Lubomír Stoklásek'), ('occupation', 'http://www.wikidata.org/entity/Q2462658')]
[('p', 'http://www.wikidata.org/entity/Q105753880'), ('name', 'Mateo Egea'), ('occupation', 'http://www.wikidata.org/entity/Q82955')]
[('p', 'http://www.wikidata.org/entity/Q107520918'), ('name', 'Jerónimo Latorre'), ('occupation', 'http://www.wikidata.org/entity/Q215793')]
[('p', 'http://www.wikidata.

Final query for this task

In [49]:
# write your queries
# wd:Q5
# wdt:P31	instance of
# wdt:P106	occupation

queryString = """
SELECT DISTINCT ?p ?name ?occupation ?occupationName
WHERE{

    {
    SELECT DISTINCT ?occupation ?occupationName
    WHERE {
       # bind something
       ?p wdt:P31 wd:Q5 ;
            wdt:P106 ?occupation.

       # get the label
       ?p sc:name ?name.
       ?occupation sc:name ?occupationName.
       
    }
    LIMIT 7000
    }
    
    filter regex(?occupationName, "archaeologist").
}
"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q3621491'), ('occupationName', 'archaeologist')]
[('occupation', 'http://www.wikidata.org/entity/Q52231239'), ('occupationName', 'medieval archaeologist')]
[('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('occupation', 'http://www.wikidata.org/entity/Q25915497'), ('occupationName', 'archaeologist of the Roman provinces')]
[('occupation', 'http://www.wikidata.org/entity/Q26261971'), ('occupationName', 'christian archaeologist')]
[('occupation', 'http://www.wikidata.org/entity/Q51034797'), ('occupationName', 'computational archaeologist')]
[('occupation', 'http://www.wikidata.org/entity/Q26424344'), ('occupationName', 'Near Eastern archaeologist')]
[('occupation', 'http://www.wikidata.org/entity/Q25931036'), ('occupationName', 'zooarchaeologist')]
[('occupation', 'http://www.wikidata.org/entity/Q54985870'), ('occupationName', 'biblical archaeologist')]
[('occupation', 'htt

## Task 5
For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

In [ ]:
#Q28171280 'ancient civilization'
#'http://www.wikidata.org/entity/Q3621491'), ('occupationName', 'archaeologist')]

# Q3621491 'archaeologist'
# Q8463726  'Chengzuo Shang'
# Q11768  'Ancient Egypt' 
# Q134178'), ('name', 'Minoan civilization')
queryString = """

    SELECT DISTINCT ?obj ?objName ?p ?name
    WHERE {
      
    ?obj ?p wd:Q134178 .

       # get the label
       
       ?obj sc:name ?objName.
       ?p sc:name ?name.
       
    }
    LIMIT 100
    
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q113937'), ('objName', 'Fritz Schachermeyr'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('obj', 'http://www.wikidata.org/entity/Q3698592'), ('objName', 'Minoan chronology'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('obj', 'http://www.wikidata.org/entity/Q16332795'), ('objName', 'Vaphio gold cups'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('obj', 'http://www.wikidata.org/entity/Q6700917'), ('objName', 'Harvester vase'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('obj', 'http://www.wikidata.org/entity/Q3566545'), ('objName', 'Bull-Leaping Fresco'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('obj', 'http://www.wikidata.org/entity/Q96749917'), ('objName', 'Akrotiri Frescoes'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('obj', 'http://www.wikidata.org/enti

In [4]:
# P101 'field of work'
# Q3621491 'archaeologist'
# wdt:P106	occupation
# P279 sub class
queryString = """

    SELECT DISTINCT  ?ancient ?ancientName ?archelogist ?archelogistName 
    WHERE {
      
      
   ?ancient wdt:P31 wd:Q28171280.
   
    ?archelogist wdt:P101 ?p.
        
       # get the label
       
       ?archelogist sc:name ?archelogistName.
       ?ancient sc:name ?ancientName.
       
    }
    LIMIT 10
    
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


Final query for this task

In [ ]:
# P101 'field of work'
# Q3621491 'archaeologist'
# wdt:P106	occupation
# P279 sub class
queryString = """

SELECT DISTINCT  ?ancient ?ancientName (count(?archelogist) as ?numAcheologist )
WHERE {
      

    ?ancient wdt:P31 wd:Q28171280.
    optional {?archelogist wdt:P101 ?ancient.}
        
       # get the label
       
       optional {?archelogist sc:name ?archelogistName.}
       ?ancient sc:name ?ancientName.
       
}
group by ?ancient ?ancientName
order by DESC (?numAcheologist)
LIMIT 100
    
"""

print("Results")
x=run_query(queryString)

## Task 6
Find the BGP for deity.

In [91]:
# wd:Q41127 Poseidon
queryString = """

    SELECT DISTINCT  ?p ?name ?obj ?objName
    WHERE {
      
       wd:Q41127  ?obj ?p.
   
       ?p sc:name ?name.
       ?obj sc:name ?objName.
       filter regex(?name, "deity").
       
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q1916821'), ('name', 'water deity'), ('obj', 'http://www.wikidata.org/prop/direct/P31'), ('objName', 'instance of')]
[('p', 'http://www.wikidata.org/entity/Q22989102'), ('name', 'Greek deity'), ('obj', 'http://www.wikidata.org/prop/direct/P31'), ('objName', 'instance of')]
2


In [92]:
# write your final query
#Q22989102 'Greek deity'
queryString = """

    SELECT DISTINCT  ?p ?name ?obj ?objName
    WHERE {
      
       wd:Q22989102  ?obj ?p.
   
       ?p sc:name ?name.
       ?obj sc:name ?objName.
       filter regex(?name, "deity").
       
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q178885'), ('name', 'deity'), ('obj', 'http://www.wikidata.org/prop/direct/P279'), ('objName', 'subclass of')]
1


Final query for this task

## Task 7
List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

In [94]:
#Q22989102 'Greek deity' P31 'instance of'
queryString = """

    SELECT DISTINCT  ?deities
    WHERE {
      
       ?deities  wdt:P31 wd:Q22989102.
   
       ?deities sc:name ?deitiesName.
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)

Results
[('deities', 'http://www.wikidata.org/entity/Q5081127')]
[('deities', 'http://www.wikidata.org/entity/Q366533')]
[('deities', 'http://www.wikidata.org/entity/Q3362052')]
[('deities', 'http://www.wikidata.org/entity/Q6612')]
[('deities', 'http://www.wikidata.org/entity/Q2144233')]
[('deities', 'http://www.wikidata.org/entity/Q1120723')]
[('deities', 'http://www.wikidata.org/entity/Q1862189')]
[('deities', 'http://www.wikidata.org/entity/Q1775444')]
[('deities', 'http://www.wikidata.org/entity/Q178417')]
[('deities', 'http://www.wikidata.org/entity/Q59697')]
[('deities', 'http://www.wikidata.org/entity/Q1084868')]
[('deities', 'http://www.wikidata.org/entity/Q1294659')]
[('deities', 'http://www.wikidata.org/entity/Q18423351')]
[('deities', 'http://www.wikidata.org/entity/Q185670')]
[('deities', 'http://www.wikidata.org/entity/Q37340')]
[('deities', 'http://www.wikidata.org/entity/Q83223')]
[('deities', 'http://www.wikidata.org/entity/Q850227')]
[('deities', 'http://www.wikidata.o

In [103]:
# wd:Q41127 Poseidon
queryString = """

    SELECT DISTINCT  ?p ?name ?obj ?objName
    WHERE {
      
       wd:Q41127 ?obj ?p.
   
       ?p sc:name ?name.
       ?obj sc:name ?objName.
       filter regex(?name, "[Oo]lympian").
       
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q101609'), ('name', 'Twelve Olympians'), ('obj', 'http://www.wikidata.org/prop/direct/P361'), ('objName', 'part of')]
1


In [105]:
# write your final query
# Q101609  'Twelve Olympians' P361 'part of'
queryString = """

    SELECT DISTINCT  ?deity ?deityName  
    WHERE {
      
       
       ?deity wdt:P361 wd:Q101609.
   
       ?deity sc:name ?deityName.
        
       
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q37340'), ('deityName', 'Apollo')]
[('deity', 'http://www.wikidata.org/entity/Q37122'), ('deityName', 'Athena')]
[('deity', 'http://www.wikidata.org/entity/Q38012'), ('deityName', 'Hera')]
[('deity', 'http://www.wikidata.org/entity/Q39503'), ('deityName', 'Artemis')]
[('deity', 'http://www.wikidata.org/entity/Q44384'), ('deityName', 'Hephaestus')]
[('deity', 'http://www.wikidata.org/entity/Q41484'), ('deityName', 'Hermes')]
[('deity', 'http://www.wikidata.org/entity/Q41127'), ('deityName', 'Poseidon')]
[('deity', 'http://www.wikidata.org/entity/Q41419'), ('deityName', 'Hestia')]
[('deity', 'http://www.wikidata.org/entity/Q34201'), ('deityName', 'Zeus')]
[('deity', 'http://www.wikidata.org/entity/Q35500'), ('deityName', 'Aphrodite')]
[('deity', 'http://www.wikidata.org/entity/Q40730'), ('deityName', 'Demeter')]
[('deity', 'http://www.wikidata.org/entity/Q40901'), ('deityName', 'Ares')]
12


## Task 8
Compare ancient Greece with ancient Egypt and find how many deities each one has.

In [107]:
# count ancient Egypt
# Q22989102  'Greek deity'
# P31 'instance of'
queryString = """

    SELECT DISTINCT  (count(?greekDeity) as ?numGreekDeity)
    WHERE {
      
       
       ?greekDeity wdt:P31 wd:Q22989102.
        
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)

Results
[('numGreekDeity', '252')]
1


In [110]:
# Q178885 'deit'
# P279 'subclass of'

queryString = """

    SELECT DISTINCT  ?Deity ?deityName
    WHERE {
      
       
       ?Deity wdt:P279 wd:Q178885.
        ?Deity sc:name ?deityName.
        filter regex(?deityName , "Egypt").
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)

Results
[('Deity', 'http://www.wikidata.org/entity/Q146083'), ('deityName', 'Ancient Egyptian deity')]
1


In [111]:
# Q146083 'Ancient Egyptian deity'
queryString = """

    SELECT DISTINCT  (count(?egyptianDeity) as ?numEgyptianDeity)
    WHERE {
      
       ?egyptianDeity wdt:P31 wd:Q146083.
        
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)

Results
[('numEgyptianDeity', '281')]
1


Final query for this task

In [117]:
# Q146083 'Ancient Egyptian deity'
# count ancient Egypt
# Q22989102  'Greek deity'
# P31 'instance of'
queryString = """

    SELECT DISTINCT  (count(?egyptianDeity) as ?numEgyptianDeity) (?numGreekDeity)
    WHERE {
      
      
       ?egyptianDeity wdt:P31 wd:Q146083.
       
        {SELECT DISTINCT   (count(?greekDeity) as ?numGreekDeity)
        WHERE {


           ?greekDeity wdt:P31 wd:Q22989102.

        }
        }
        
    }
    LIMIT 20
    
"""

print("Results")
x=run_query(queryString)

Results
[('numEgyptianDeity', '281'), ('numGreekDeity', '252')]
1
